<a href="https://colab.research.google.com/github/Zahrakh98/id2223/blob/main/Persian_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare work

View GPU specification

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Install packages

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mtooqztw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mtooqztw
  Resolved https://github.com/huggingface/transformers to commit 58e7f9bb2faf30622c9bead7adf472ac59f3d301
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8178366 sha256=83864208e3dee6d72edb646513aece162630e261b4f30f41e6568acdd83a5eb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-jqw8bz_b/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Link to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load data

Download data from huggingface

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "fa", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "fa", split="test", use_auth_token=True)


print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 6671it [00:00, 66700.36it/s]
Reading metadata...: 13342it [00:00, 60022.16it/s]
Reading metadata...: 19458it [00:00, 60508.61it/s]
Reading metadata...: 26951it [00:00, 60721.61it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9480it [00:00, 94790.60it/s]
Reading metadata...: 24401it [00:00, 108837.48it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13793it [00:00, 93989.80it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 37239
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10288
    })
})


Remove some factors

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 37239
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10288
    })
})


Save common_voice to local

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice.csv"
common_voice.save_to_disk(output_dir)

Mounted at /content/drive


Saving the dataset (0/3 shards):   0%|          | 0/37239 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10288 [00:00<?, ? examples/s]

# Prepare Feature Extractor, Tokenizer and Data

Load WhisperFeatureExtractor

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Load WhisperTokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Persian", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Combine To Create A WhisperProcessor

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Persian", task="transcribe")

Prepare Data

In [ ]:
!pip install datasets>=2.6.1

In [ ]:
from datasets import load_from_disk, DatasetDict
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice.csv"

common_voice = load_from_disk(output_dir, keep_in_memory=True)
print(common_voice["train"][0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'audio': {'path': 'common_voice_fa_18206958.mp3', 'array': array([ 0.00000000e+00,  2.39904315e-13,  3.64981957e-14, ...,
        1.68958322e-05, -6.47242086e-06, -2.39575056e-05]), 'sampling_rate': 48000}, 'sentence': 'من برات يادداشت گذاشتم'}


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': 'common_voice_fa_18206958.mp3', 'array': array([ 3.49245965e-10,  1.74622983e-10, -9.89530236e-10, ...,
       -3.93181108e-05, -9.63849016e-06,  1.93573069e-05]), 'sampling_rate': 16000}, 'sentence': 'من برات يادداشت گذاشتم'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')
train_cache_file = "/content/drive/MyDrive/train_cache_file"
validation_cache_file = "/content/drive/MyDrive/validation_cache_file"
test_cache_file = "/content/drive/MyDrive/test_cache_file"
other_cache_file = "/content/drive/MyDrive/other_cache_file"
invalidated_cache_file = "/content/drive/MyDrive/invalidated_cache_file"
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1,
                                cache_file_names={"train": train_cache_file, "validation": validation_cache_file, "test": test_cache_file, "other": other_cache_file, "invalidated": invalidated_cache_file})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/37239 [00:00<?, ? examples/s]

Map:   0%|          | 0/10288 [00:00<?, ? examples/s]

Save mapped common_voice to local

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice_mapped.csv"
common_voice.save_to_disk(output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving the dataset (0/72 shards):   0%|          | 0/37239 [00:00<?, ? examples/s]

Saving the dataset (0/20 shards):   0%|          | 0/10288 [00:00<?, ? examples/s]

# Training preparation

In [8]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets>=2.6.1
!pip install evaluate>=0.30
!pip install jiwer

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-wa05htcf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-wa05htcf
  Resolved https://github.com/huggingface/transformers.git to commit 80377eb018c077dba434bc8e7912bcaed3a64d09
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8195352 sha256=366cc4b294b3939aee0f16dc34bfb233e090e1b182d1bcf393a4816ad9ef377f
  Stored in directory: /tmp/pip-ephem-wheel-cache-hwi9j__m/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-hv11nr4p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-hv11nr4p
  Resolved https://github.com/huggingface/accelerate.git to commit 9964f90fd7d50577998a22f3dba8590e644d255b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.25.0.dev0-py3-none-any.whl size=267070 sha256=4f06ccd82f7b043c8e9fa3e1c70b1e9cd81ebdc45541e8d3e7c551a1a1c354c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-j2ra5s17/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.4 MB/s eta 0:00:00


Load WhisperTokenizer

In [9]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Persian", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Combine To Create A WhisperProcessor

In [10]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Persian", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Load mapped common_voice

In [11]:
from datasets import load_from_disk, DatasetDict
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice_mapped.csv"

common_voice = load_from_disk(output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define a Data Collator

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Initialize the collator

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Evaluation Metrics

In [14]:
import evaluate

metric = evaluate.load("wer")

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Load a Pre-Trained Checkpoint

In [16]:


from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [17]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

Define the Training Configuration

In [21]:
!pip install accelerate -U
!pip install transformers[torch]

from transformers import Seq2SeqTrainingArguments
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/check_points"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=600,
    max_steps=600,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy="steps",
    save_steps=200, # checkpoint every 400 steps
    eval_steps=200, # evaluate every 400 steps
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_strategy="checkpoint"
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ImportError: ignored

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

# Training

In [15]:
import os
from google.colab import drive
drive.mount('/content/drive')
last_checkpoint_path = "/content/drive/MyDrive/check_points/checkpoint-400"

if os.path.exists(last_checkpoint_path):
    trainer.train(resume_from_checkpoint=last_checkpoint_path)
    print("Resume from last checkpoint and continue training.")
else:
    trainer.train()
    print("Start training.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
400,0.389200,0.626038,61.157438


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Resume from last checkpoint and continue training.


In [16]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: fa, split: test",
    "language": "fa",
    "model_name": "Whisper Small fa",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [17]:
trainer.push_to_hub(**kwargs)

'https://huggingface.co/zahrakh98/check_points/tree/main/'

# New Section

Gradio

In [1]:
!pip install moviepy librosa
!pip install gradio torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.4 MB/

In [2]:
!pip install fastapi==0.103.2
!pip install gradio==3.48.0
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.104.1
    Uninstalling fastapi-0.104.1:
      Successfully uninstalled fastapi-0.104.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 31.8 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.7.1
    Uninstalling gradio_client-0.7.1:
      Successfully uninstalled gradio_client-0.7.1
  Attempting uninstall: gradio
    Found existing installation: gradio 4.8.0
    Uninstalling gradio-4.8.0:
      Successfully uninstalled gradio-4.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import pipeline
import gradio as gr
import os
import pytube as pt
import subprocess
import sys

youtube_file_path = "youtube_audio"

pipe = pipeline(model="zahrakh98/check_points")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

def _return_yt_html_embed(yt_url):
    video_id = yt_url.split("?v=")[-1]
    HTML_str = (
        f'<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/{video_id}"> </iframe>'
        " </center>"
    )
    return HTML_str

def transcribe_youtube(yt_url):
    yt = pt.YouTube(yt_url)
    html_embed_str = _return_yt_html_embed(yt_url)
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename=youtube_file_path)
    text = transcribe(youtube_file_path)
    return text

def convert_video_to_audio_ffmpeg(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"

def transcribe_video(video):
    audio_filename = convert_video_to_audio_ffmpeg(video)
    text = transcribe(audio_filename)
    return text

iface = gr.Blocks()

microphone_trans = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs="text",
    title="Whisper Persian",
    description="Realtime demo for Persian speech recognition using a fine-tuned Whisper small model.",
)

youtube_trans = gr.Interface(
    fn=transcribe_youtube,
    inputs=gr.Textbox(lines=1, placeholder="Paste the URL to a video here", label="video url"),
    outputs="text",
    title="Whisper Persian",
    description="Realtime demo for Persian speech recognition using a fine-tuned Whisper small model.",
)

video_trans = gr.Interface(
    fn=transcribe_video,
    inputs=gr.Video(sources="upload", type="filepath"),
    outputs="text",
    title="Whisper Persian",
    description="Realtime demo for Persian speech recognition using a fine-tuned Whisper small model.",
)

with iface:
    gr.TabbedInterface([microphone_trans, youtube_trans, video_trans], ["Transcribe Microphone", "Transcribe Youtube Video", "Transcribe Video"])
iface.launch(debug=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
<ipython-input-5-2ee6943a79fa>:48: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': 'microphone'}
  inputs=gr.Audio(sources="microphone", type="filepath"),
<ipython-input-5-2ee6943a79fa>:64: GradioUnusedKwargWarning: You have unused kwarg parameters in Video, please remove them: {'sources': 'upload', 'type': 'filepath'}
  inputs=gr.Video(sources="upload", type="filepath"),
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8cf43c5542183bcfbe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:
from transformers import pipeline
import gradio as gr
import requests
import re

file_path = "/content/drive/MyDrive/videos"
filename = "douyin_video"

pipe = pipeline(model="zahrakh98/check_points")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

def download_video(video_url):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.25 Safari/537.36 Core/1.70.3877.400 QQBrowser/10.8.4506.400'}
    response = requests.get(url=video_url, headers=headers)
    if response and response.status_code == 200:
        if not os.path.exists(file_path):
            os.makedirs(file_path)
        with open(file_path + "/" + filename, 'wb') as f:
            f.write(response.content)
    return filename

def transcribe_video(video_url):
    filename = download_video(video_url)
    return filename

iface = gr.Blocks()

microphone_trans = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath", optional=True),
    outputs="text",
    title="Whisper Persian",
    description="Realtime demo for Persian speech recognition using a fine-tuned Whisper small model.",
)

video_trans = gr.Interface(
    fn=transcribe_video,
    inputs=gr.Textbox(lines=1, placeholder="Paste the URL to a video here", label="video url"),
    outputs="text",
    title="Whisper Persian ",
    description="Realtime demo for Persian speech recognition using a fine-tuned Whisper small model.",
)

with iface:
    gr.TabbedInterface([microphone_trans, video_trans], ["Transcribe Microphone", "Transcribe Video"])
iface.launch(debug=True)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
<ipython-input-4-441aece34290>:33: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.Audio(source="microphone", type="filepath", optional=True),
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5859b9ca932d5ddfeb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5859b9ca932d5ddfeb.gradio.live
